In [1]:
import pandas as pd

data = pd.read_excel("ccapm.xlsx", header=None)
data.columns = ["cratio", "rrate", "e"]

print(data.head())
print(data.columns)


     cratio     rrate         e
0  1.010612  1.006276  0.034638
1  0.991563  1.035976  0.106118
2  1.009609  1.018062 -0.084368
3  0.999425  0.996259  0.096085
4  0.996430  1.034679 -0.005752
Index(['cratio', 'rrate', 'e'], dtype='object')


In [2]:
# Create lagged instruments
data["cratio_lag"] = data["cratio"].shift(1)
data["rrate_lag"]  = data["rrate"].shift(1)
data["e_lag"]      = data["e"].shift(1)

# Drop the first row (t = 0) because of missing lags
data_gmm = data.dropna().copy()

data_gmm.head()

,cratio,rrate,e,cratio_lag,rrate_lag,e_lag
1,0.991563,1.035976,0.106118,1.010612,1.006276,0.034638
2,1.009609,1.018062,-0.084368,0.991563,1.035976,0.106118
3,0.999425,0.996259,0.096085,1.009609,1.018062,-0.084368
4,0.996430,1.034679,-0.005752,0.999425,0.996259,0.096085
5,1.002306,0.986218,-0.035698,0.996430,1.034679,-0.005752


In [3]:
import numpy as np

# Work on the cleaned dataset with lags
T = len(data_gmm)

# y_t = 1 for all t
y = np.ones(T)

# X_t = [cratio_t, rrate_t]
X = data_gmm[["cratio", "rrate"]].to_numpy()

# Instruments Z_t = [1, cratio_{t-1}, rrate_{t-1}, e_{t-1}]
Z = np.column_stack([
    np.ones(T),
    data_gmm["cratio_lag"].to_numpy(),
    data_gmm["rrate_lag"].to_numpy(),
    data_gmm["e_lag"].to_numpy()
])

print("y shape:", y.shape)
print("X shape:", X.shape)
print("Z shape:", Z.shape)

y shape: (237,)
X shape: (237, 2)
Z shape: (237, 4)


In [4]:
def ccamp_func(params, exog):
    beta, gamma = params
    cratio_t = exog[:, 0]
    rrate_t  = exog[:, 1]
    return beta * (cratio_t ** (-gamma)) * rrate_t

In [5]:
from statsmodels.sandbox.regression.gmm import NonlinearIVGMM
import numpy as np

# Define the model
model = NonlinearIVGMM(endog=y, exog=X, instrument=Z, func=ccamp_func)

# Estimate parameters with starting values beta=1, gamma=1
start_params = np.array([1.0, 1.0])

res = model.fit(start_params=start_params, maxiter=10)  # iterated GMM

print("Estimated parameters (beta, gamma):", res.params)
print("Std errors:", res.bse)
print(res.summary())

Optimization terminated successfully.
         Current function value: 0.000014
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005436
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.005188
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.005197
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.005196
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Estimated parameters (beta, gamma): [0.9977703  0.77231495]
Std errors: [0.00432883 1.77044988]
                            NonlinearIVGMM Results       

In [6]:
# Newey–West / HAC GMM with truncation lag 5
res_hac = model.fit(
    start_params=start_params,
    maxiter=10,                  # still do iterated GMM
    weights_method="hac",        # <– use HAC / Newey–West
    wargs={"maxlag": 5}          # <– truncation lag = 5
)

print(res_hac.summary())

print("HAC estimates (beta, gamma):", res_hac.params)
print("HAC std errors:", res_hac.bse)


Optimization terminated successfully.
         Current function value: 0.000014
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.008062
         Iterations: 5
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.007271
         Iterations: 4
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.007278
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
Optimization terminated successfully.
         Current function value: 0.007277
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3
                            NonlinearIVGMM Results                            
Dep. Variable:                      y   Hansen J:                        1

In [7]:
import scipy.stats as st

# Extract estimate and HAC standard error of beta
beta_hat = res_hac.params[0]
se_beta  = res_hac.bse[0]

# Null hypothesis value
beta_null = 0.98

# Compute z-statistic
z_stat = (beta_hat - beta_null) / se_beta

# Two-sided p-value
p_value = 2 * (1 - st.norm.cdf(abs(z_stat)))

print("beta_hat =", beta_hat)
print("SE(beta_hat) =", se_beta)
print("z-statistic =", z_stat)
print("p-value =", p_value)

beta_hat = 0.9979139993610195
SE(beta_hat) = 0.0043997650811589675
z-statistic = 4.071580875472726
p-value = 4.669514382982243e-05
